# Vector spatial join processing task

In this notebook, we will demonstrate joining a vector layer of points to another vector layer containing a road network using nearest neighbours. 

In many transport logistics problems, locations to be visited first need to be snapped to the road network. Here, a power company needs to find the closest road to each power substation in order to find a route in the road network.

Datasets used:

- [Ordnance Survey Open Roads](https://beta.ordnancesurvey.co.uk/products/os-open-roads) - road links filtered to the SP 100x100km grid square `51.724598,-2.001086,52.58593,-0.52677` roughly including half of Birmingham, Coventry, Oxford and Milton Keynes. Converted to EPSG:4326 and to `geojson` resulting in ~183 MB.
- [OpenStreetMap power substations](https://wiki.openstreetmap.org/wiki/Tag:power%3Dsubstation) in SP grid downloaded using the [API](https://overpass-turbo.eu/) using the query `node[power=substation](51.724598,-2.001086,52.58593,-0.52677);out;`. < 1 MB

## 0. Install Python packages

In [ ]:
!pip install geopandas folium

## 1. Load data

First connect Databricks to your datalake.

In [ ]:
datalake = "/dbfs/mnt/copgeospatial"

In [ ]:
import pandas as pd 
import geopandas as gpd

Roads dataset `geodataframe`. Geometry is `LineString`, 255946 rows. Columns include `gml_id` (primary key), along with road details such as `roadClassification`, `roadFunction`, `formOfWay`, `length`.

In [ ]:
gdf_roads = gpd.read_file(f"{datalake}/OS_OpenRoads_SP_RoadLink_4326.geojson")
gdf_roads.head()

Power substations `geodataframe`. Geometry is `Point`, 1238 rows. Columns include `id` (primary key), and OpenStreetMap tags for substations such as `name`, `operator`, `voltage` etc.

In [ ]:
gdf_substations = gpd.read_file(f"{datalake}/SP_Power_Substations.geojson")
gdf_substations.head()

## 2. Process data

Perform a spatial join using native [geopandas](https://geopandas.org/en/latest/docs/user_guide/mergingdata.html#spatial-joins) methods. Note that because the UK is small, we can approximately perform planar distance calculations despite both `geodataframe`s being in an unprojected geographic CRS `EPSG:4326`. To do this, we can convert a distance in metres to a degree "distance" by dividing by `R * pi / 180` where `R=6371000` is the radius of the Earth.

In [ ]:
gdf_roads_substations = gdf_substations.sjoin_nearest(
    gdf_roads, 
    how="left",
    max_distance=100/111195 #100 metres in units of degrees
).merge(
    gdf_roads, 
    on="gml_id", 
    suffixes=("_delete", None)
)

## 3. Visualise data on map

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[54.44, -3.5], width=500, height=750, zoom_start=12, tiles="CartoDB positron")

m = gdf_roads_substations.explore(
    m=m,
    tooltip=["gml_id", "roadFunction", "formOfWay", "length"]
)

for i, row in gdf_substations.iterrows():
    folium.Circle(
        location=(row.geometry.y, row.geometry.x), 
        radius=30, 
        color="crimson",
        fill=True,
        tooltip=f"Substation name {row['name']}"
    ).add_to(m)

folium.LayerControl().add_to(m)
m.fit_bounds(m.get_bounds()) 

In [ ]:
m

In [ ]:
m.save('/dbfs/mnt/copgeospatial/02_output.html')

### Display map

In [1]:
from IPython.display import IFrame
IFrame("02_output.html", width="100%", height="700")